In [145]:
import numpy as np
import random
import pandas as pd
import os

In [146]:
# Agent
class Agent(object):

    def __init__(self):
        
        
        self.player = 1 # player 1 = 1 and player 2 = -1
        
        self.number_match = 0
        
        self.results ={
            'win':0,
            'draw':0,
            'lost':0}
        
        self.Q_table = {
            'states' : [],
            'actions': ['(0, 0)','(0, 1)','(0, 2)','(1, 0)','(1, 1)','(1, 2)','(2, 0)','(2, 1)','(2, 2)'],
            'Q': []
        }
        
        self.path = {
            'states':  [], # boards
            'actions': [], # posição no tabuleiro
        }
        
    def reset_game(self):
        self.player = 1
        self.path = {
            'states': [],
            'actions':[],
        }
    
    def reset_historic_game(self):
        self.results ={
            'win':0,
            'draw':0,
            'lost':0}
        
    def save_result(self, resultado):
        
        if resultado == 1:
            #print('venceu')
            self.results['win'] += 1
            
        elif resultado == -1:
            #print('perdeu')
            self.results['lost'] += 1

        else:
            #print('empate')
            self.results['draw'] += 1
            
    def Q_table_df(self):
        
        df = pd.DataFrame(
            index= self.Q_table['states'],
            columns= self.Q_table['actions'],
            data = self.Q_table['Q']
            )
            #data = 0 )
        return df
    
    def update_Q(self, reward):
        
        # Q(s,a) = Q(s,a) + alpha* ( R(s) + * Gamma * max_Q(s+1,:) - Q(s,a) ) )
        # R(s) = Reward...
        
        lr =    0.9 # Alpha - Taxa de Aprendizagem
        gamma = 0.9 # Gamma - Fator de Desconto
        
        # Lista de Estados e Ações - Executados
        states_actions = list( self.path.values() )

        # Lista de Estados Reverso (pois iremos do FUTURO pro PASSADO)
        states =  list( reversed( states_actions[0] ) )

        # Lista de Ações Reverso   (pois iremos do FUTURO pro PASSADO)
        actions = list( reversed( states_actions[1] ) )

        # Marcador para eu saber onde estou
        index = 0
        for s2, a2 in zip( states, actions ):
            
            
            if reward >= 0: 

                try:
                    # index  = 0 é a ultima ação que levou a vitóriam, ou derrota
                    if index == 0:

                        s2 = self.Q_table['states'].index(str(s2))
                        a2 = self.Q_table['actions'].index(str(a2))

                        self.Q_table['Q'][s2][a2] = lr* ( reward ) #self.Q_table['Q'][s2][a2] = reward 


                        # Fazer o mesmo, mas agora para o States adiantado

                        ##### Next Value #####

                        # ESTADO avançado
                        index += 1

                        s1 = states[index]
                        s1 = self.Q_table['states'].index(str(s1))

                        a1 = actions[index]
                        a1 = self.Q_table['actions'].index(str(a1))

                        # a2 -> deixa em aberto, por que estamos interessado na ação com valor MÁXIMO do respectivo ESTADO avançado Max_Q(s+1,:)
                        self.Q_table['Q'][s1][a1] += lr*( 0 + gamma*np.max( self.Q_table['Q'][s2] ) - self.Q_table['Q'][s1][a1] )

                    else:

                        ##### pegar o index numérico dos States e Actions
                        s2 = self.Q_table['states'].index(str(s2))
                        a2 = self.Q_table['actions'].index(str(a2))


                        # Fazer o mesmo, mas agora para o States adiantado

                        ##### Next Value #####

                        # ESTADO avançado
                        index += 1

                        s1 = states[index]
                        s1 = self.Q_table['states'].index(str(s1))

                        a1 = actions[index]
                        a1 = self.Q_table['actions'].index(str(a1))

                        # a2 -> deixa em aberto, por que estamos interessado na ação com valor MÁXIMO do respectivo ESTADO avançado Max_Q(s+1,:)
                        self.Q_table['Q'][s1][a1] += lr*( 0 + gamma*np.max( self.Q_table['Q'][s2] ) - self.Q_table['Q'][s1][a1] ) 

                # Não há mais Estados Adiantados para buscar.   
                except IndexError:
                    continue
            
            
            if reward < 0:
            # Se for negativo tem que DESCONTAR, pra isso, usa-se o MIN_Q
                
                try:
                    # index  = 0 é a ultima ação que levou a vitóriam, ou derrota
                    if index == 0:

                        s2 = self.Q_table['states'].index(str(s2))
                        a2 = self.Q_table['actions'].index(str(a2))

                        self.Q_table['Q'][s2][a2] = lr* ( reward ) #self.Q_table['Q'][s2][a2] = reward 


                        # Fazer o mesmo, mas agora para o States adiantado

                        ##### Next Value #####

                        # ESTADO avançado
                        index += 1

                        s1 = states[index]
                        s1 = self.Q_table['states'].index(str(s1))

                        a1 = actions[index]
                        a1 = self.Q_table['actions'].index(str(a1))

                        # a2 -> deixa em aberto, por que estamos interessado na ação com valor MÁXIMO do respectivo ESTADO avançado Max_Q(s+1,:)
                        self.Q_table['Q'][s1][a1] += lr*( 0 + gamma*np.min( self.Q_table['Q'][s2] ) - self.Q_table['Q'][s1][a1] )

                    else:

                        ##### pegar o index numérico dos States e Actions
                        s2 = self.Q_table['states'].index(str(s2))
                        a2 = self.Q_table['actions'].index(str(a2))


                        # Fazer o mesmo, mas agora para o States adiantado

                        ##### Next Value #####

                        # ESTADO avançado
                        index += 1

                        s1 = states[index]
                        s1 = self.Q_table['states'].index(str(s1))

                        a1 = actions[index]
                        a1 = self.Q_table['actions'].index(str(a1))

                        # a2 -> deixa em aberto, por que estamos interessado na ação com valor MÁXIMO do respectivo ESTADO avançado Max_Q(s+1,:)
                        self.Q_table['Q'][s1][a1] += lr*( 0 + gamma*np.min( self.Q_table['Q'][s2] ) - self.Q_table['Q'][s1][a1] ) 

                # Não há mais Estados Adiantados para buscar.   
                except IndexError:
                    continue   

In [165]:
# Enviroment
class Enviroment(Agent):

    def __init__(self, epsilon):
        
        super().__init__()
        
        
        self.epsilon = epsilon # utilizo para treinar
        
        

        # Board (é nosso ESTADO ATUAL)
        self.board = np.zeros((3,3))

        # Set of States
        self.states = []
        
        # pos jogada
        self.pos = 0

    def reset_game(self):
        self.board = np.zeros((3,3))

    # Plotar o Board
    def draw_board(self):

        draw = ''

        for i in range(3):
            for j in range(3):
                simbolo = ''
                # simbolo X (p1 = 1) ou O (p2 = -1)
                if self.board[i][j] == 1:
                    symbol = 'X'
                elif self.board[i][j] == -1:
                    symbol = 'O'
                else:
                    symbol = ' '


                draw += '|'+symbol+''



                if j == 2:

                    draw +='|\n-------\n'

        print(draw)

    # Posições disponíveis
    def available_moves(self):
        return np.argwhere(self.board == 0)
    # Jogar uma posição disponível
    def available_move_choice(self):
        return random.choice(self.available_moves())

    # Checar Resultado    
    def check_result(self):

        # Row
        if sum(self.board[0]) == 3 or sum(self.board[1]) == 3 or sum(self.board[2]) == 3:
            #print('venceu')
            return 1
        if sum(self.board[0]) == -3 or sum(self.board[1]) == -3 or sum(self.board[2]) == -3:
            #print('perdeu')
            return -1
        # Col
        if sum(self.board[:,0]) == 3 or sum(self.board[:,1]) == 3 or sum(self.board[:,2]) == 3:
            #print('venceu')
            return 1
        if sum(self.board[:,0]) == - 3 or sum(self.board[:,1]) == - 3 or sum(self.board[:,2]) == - 3:
            #print('perdeu')
            return -1
        # Diagonal
        if sum(self.board.diagonal()) == 3 or sum(np.fliplr(self.board).diagonal()) == 3:
            #print('venceu')
            return 1
        if sum(self.board.diagonal()) == -3 or sum(np.fliplr(self.board).diagonal()) == -3:
            #print('perdeu')
            return -1
        # Empate
        if not 0 in self.board:
            #print('empate')
            return 0

        #########################################################
        ## continua = 2, empate = 0, vitoria = 1, derrota = -1 ##
        #########################################################

        return 2

    # Dar recompensa        
    def reward(self, result):

        if result == 1:  # Vitória
            return 1

        if result == -1: # Derrota
            return -1

        if result == 0:  # Empate
            return 0.001
    
    
    # jogada - Random 
    def select_pos_by_random(self, player, name):
        
        row_col = self.available_move_choice()
        
        row = row_col[0] # Linha
        col = row_col[1] # Coluna

        self.board[row][col] = player
        
        self.pos = row,col
        
        #print(name + f' jogou na posição { str(self.pos) }')
           
    # jogada - humano   
    def select_pos_by_input(self, player, name):
        
        # desenhar jogada do player 
        self.draw_board()
        while True:
            row = int( input('Row: ') )
            col = int( input('Col: ') )
            
            if [row,col] in self.available_moves().tolist(): # Refransforme Em lista... Array ele aceita 
                
                self.board[row][col] = player
                self.pos = row,col
                break
            else:
                input('try other position...')
    

    def select_pos_by_Q(self,player, name):

        # Veja o estado atual seu (Seu board)... pegue a ação com maior Q


        #epsilon = 0.05
        
        # jogada Aleatória
        if np.random.uniform(0, 1) < self.epsilon:
            
            #print('********jogada aleatória - Caiu no EPSILON ***********')
            
            self.select_pos_by_random( self.player, name = 'player '+str(self.player) )


            #print('usando aleatório')

        # Vai na tabela e joga
        else:

            # Se existir esse estado gravado...

            if str(self.board) in self.Q_table['states']:


                #print('usando o Q')


                index_state = self.Q_table['states'].index( str(self.board) )
                #index_action= self.Q_table['Q'][index_state].index( str(np.max(self.Q_table['Q'][index_state])) )
                #index_qmax = np.argmax(self.Q_table['Q'][index_state])


                # pega todos valores de Q com respectivo index state na ordem DESCRESCENTE
                # assim, se a posição máx já estiver ocupada, ele vai pro segundo maior e assim por diante.

                #print(sorted( self.Q_table['Q'][index_state], reverse = True ) )
                #input()
                

                # pega o maior na ordem decrescente... 
                for qmax in sorted( self.Q_table['Q'][index_state], reverse = True ):
                    
                    # logo se for Zero não temos estado treinado
                    if qmax == 0:
                        
                        #print(f'********Jogada Aleatório - qmax = {qmax} ... não tem treino***********')
                        
                        self.select_pos_by_random( self.player, name = 'player '+str(self.player) )
                        break


                    index_qmax = self.Q_table['Q'][index_state].index( qmax )

                    action = self.Q_table['actions'][index_qmax]

                    row = int(action[1:2])
                    col = int(action[4:5])

                    if [row,col] in self.available_moves().tolist(): # Refransforme Em lista... Array ele aceita  

                        self.board[row][col] = player

                        self.pos = row,col
                        
                        #print(f'******** Jogada Inteligente - melhor Q:{qmax}***********')


                        break



            # se não existir, joga aleatório mesmo
            else:
                
                #print('********Jogada Aleatória - Não existe este Estado***********')
                
                #print(str(self.board))
                
                self.select_pos_by_random( self.player, name = 'player '+str(self.player) )

In [166]:
# funct to start the game
def start():
    while True:

        ##################### Criação da Tabela Q (antes) ###################
        # Se não existe este Estado dentro da Tabela Q, adicione
        if str(env.board) not in agent.Q_table['states']:

            # 1-) Adicionar Estado Atual
            agent.Q_table['states'].append( str(env.board ) )

            # 2-) Add valor de Q
            agent.Q_table['Q'].append( [0,0,0,0,0,0,0,0,0] )
        ###############################################################


        # Registrar o State Inicial no PATH
        agent.path['states'].append( str(env.board) )


        ############################ Agente Executa Ação no Ambiente #################### 
        if agent.player == 1: # PLAYER 1
            env.select_pos_by_Q( agent.player, name = 'player '+str(agent.player) )

        else:               # PLAYER 2 
            env.select_pos_by_random( agent.player, name = 'player '+str(agent.player) )
        #################################################################################


        # Registrar o Action realizada no PATH
        agent.path['actions'].append( str(env.pos) )



        # ( Desenha  Board )
        #env.draw_board()

        ########################## Ambiente Responde ######################################
        # checa resultado
        if env.check_result() != 2: # continua = 2, empate = 0, vitoria = 1, derrota = -1

            # resultado do jogo
            agent.save_result( env.check_result() )

            # Valor da Recompensa
            reward = env.reward( env.check_result() )

            # Update Q Table
            agent.update_Q( reward )

            # Reset Game
            env.reset_game()
            agent.reset_game()
            
            # add partida jogada
            agent.number_match += 1

            break
            


        # Mudar jogador    
        agent.player *= -1 # switch players

In [167]:
agent = Agent()
env = Enviroment(epsilon=0.0)
env.Q_table = agent.Q_table

In [163]:
# Fazer Load dos dados já treinados

with open('./Q_table.pkl', 'rb') as handle:
    Q_table = pickle.load(handle)
with open('./partidas.pkl', 'rb') as handle:
    number_match = pickle.load(handle)

agent.number_match = number_match
agent.Q_table = Q_table 

In [158]:
for i in range(1):
    start()

In [168]:
agent.number_match

83044

In [169]:
# Cada época
for k in range(10):
    
    #  Treina 100 x
    for i in range(1000):
        start()
    print(str(agent.results) + " partidas jogadas: " + str(agent.number_match) )
    agent.reset_historic_game()

{'win': 975, 'draw': 13, 'lost': 12} partidas jogadas: 84044
{'win': 985, 'draw': 10, 'lost': 5} partidas jogadas: 85044
{'win': 985, 'draw': 11, 'lost': 4} partidas jogadas: 86044
{'win': 983, 'draw': 7, 'lost': 10} partidas jogadas: 87044
{'win': 988, 'draw': 7, 'lost': 5} partidas jogadas: 88044
{'win': 969, 'draw': 14, 'lost': 17} partidas jogadas: 89044
{'win': 973, 'draw': 11, 'lost': 16} partidas jogadas: 90044
{'win': 986, 'draw': 5, 'lost': 9} partidas jogadas: 91044
{'win': 975, 'draw': 12, 'lost': 13} partidas jogadas: 92044
{'win': 986, 'draw': 9, 'lost': 5} partidas jogadas: 93044


In [170]:
# SALVAR 
import pickle

with open("./Q_table.pkl", "wb") as tf:
    pickle.dump(agent.Q_table,tf)
    
with open("./partidas.pkl", "wb") as tf:
    pickle.dump(agent.number_match,tf)